<a href="https://colab.research.google.com/github/SIDIBEMoussa/Application-de-HMM/blob/main/TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP2

# Introduction

- Le but de ce TP est de **se familiariser avec les concepts** des **`modèles de Markov cachés`** (**= HMM**).
- La portée de l'ambition est donc limitée et les exemples sont très simples.

Sujets principaux:
- Formulation du problème à l'aide d'un exemple de base pour introduire la **terminologie HMM**.
- Illustration de la **règle de Bayes** pour déterminer l'état caché le plus probable compte tenu d'une observation.
- Implémentation du **Forward Algorithm** et du **Backward Algorithm** pour calculer la probabilité d'une séquence d'observation particulière.
- Définition et illustration des bénéfices de la **Programmation Dynamique**.
- Implémentation de l'**Algorithme de Décodage de Viterbi** pour trouver la séquence d'états cachés la plus probable étant donné une séquence d'observation.
- Implémentation de l'**algorithme de Baum-Welch** pour trouver les paramètres HMM les plus probables compte tenu de certaines séquences d'observation.


# Motivation du problème

- Votre voiture roule sur une **autoroute à 2 voies**.
- Imaginez que vous puissiez **suivre à distance la vitesse de la voiture** (par exemple, elle est communiqué).
- Mais vous n'avez **pas d'accès direct à la position latérale** ('voie de droite' de 'voie de gauche').
- Formellement, vous **ne pouvez pas observer directement la marche stochastique sous-jacente entre les états de « voie »**.
- Comment pouvez-vous **déduire la "voie"** en vous basant sur les informations uniques que vous recevez (la "vitesse") ?
### Probabilité d'émission

Si je vous dis que je conduis à « basse vitesse », vous **pouvez deviner** que je suis sur la voie de droite.
- Par exemple, parce que je conduis seul à un rythme raisonnable.
- Soit parce que je suis bloqué par un véhicule lent alors que je n'arrive pas à le reprendre.
- Mais je pourrais aussi rouler vite sur cette « voie de droite »

De même, si vous êtes informé d'une « vitesse élevée », vous pourriez dire que je suis **plus susceptible** de conduire sur la voie de gauche.
- Probablement en dépassant un autre véhicule.
- Néanmoins, ce n'est **pas toujours vrai** : Pensez à la situation où vous attendez sur la voie de gauche derrière un camion essayant de dépasser un autre camion.

On obtient ici une **première intuition** :
- La variable "voie" semble avoir un impact sur la variable "vitesse".
- En d'autres termes : **vous ne roulez pas au même rythme selon que vous êtes sur la « voie de gauche » ou la « voie de droite »**.
- Mais la relation n'est **pas déterministe**. C'est **stochastique**.

Ce **résultat de causalité** sera modélisé à l'aide de **`probabilités d'émission`** dans ce qui suit.

### Probabilité de transition

Vous pourriez avoir une deuxième intuition sur le **processus séquentiel** :
- Les conducteurs humains **restent généralement sur leurs voies**.
- Par conséquent, si vous êtes sur la « voie de droite » au temps « t », vous êtes probablement toujours sur la « voie de droite » au temps « t+1 ».
- Encore une fois, cela **ne tient pas toujours** et vous pouvez trouver des **exceptions**.
- Mais voici une seconde intuition : **la `voie` à l'instant `t` est influencée par la `voie` à l'instant `t-1`**.

Le concept de **`probabilité de transition`** sera utilisé pour modéliser cette seconde remarque.


## Terminologie

Dans les sections suivantes, nous verrons comment pouvons-nous **soutenir mathématiquement ces intuitions**.

| ![La `vitesse` est `l'observation` tandis que la `voie` constitue `l'état caché`. Quelques exemples montrent que toutes les `émissions` sont possibles.](docs/terminology.PNG "La `vitesse` est l `observation` tandis que la `voie` constitue `l'état caché`. Certains exemples montrent que toutes les `émissions` sont possibles .") |
|:--:|
| *La `vitesse` est `l'observation` tandis que la `voie` constitue `l'état caché`. Quelques exemples montrent que toutes les `émissions` sont possibles.* |




## Objectifs

Nous pouvons maintenant définir trois problèmes qui peuvent être résolus par un HMM :

- 1- **Apprentissage**
	- Le premier est **l'apprentissage des paramètres** de sa **structure latente** (modèle 'émission', modèle 'transition' et distribution 'état initial').
	- Dans le cas d'une structure connue et d'un **échantillonnage pleinement observable**, on peut appliquer le concept d'**Estimation de Vraisemblance Maximale** (MLE) :
		- Certaines séquences d'observation (`vitesse`) et leurs états associés (`voie`) ont été collectés. Les **échantillons** forment le **ensemble de données d'entraînement**.
		-  Les paramètres peuvent être sélectionnés de manière à **maximiser la probabilité** pour le modèle d'**avoir produit les données** à partir de l'ensemble de données donné.
		-  La question [Q1](#q1) introduit cette méthode d'**apprentissage supervisé**.
	- S'il n'est **pas possible d'échantillonner à partir d'états cachés**, optez pour l'**apprentissage non supervisé**.
		- Une méthode basée sur la **Maximisation des attentes** (EM) est présentée dans [Q6](#q6).

- 2- **Évaluation**
	- Une fois la structure du HMM définie et ses paramètres déterminés, la deuxième tâche consiste à trouver ** quelle est la probabilité qu'il obtienne une séquence d'observation particulière **.
	- Ce problème, parfois appelé **"Scoring"**, est traité à la question [Q3](#q3).

- 3- **Inférence**
	- Dans le troisième problème, nous voulons **déduire la séquence de voies** parcourues par la voiture ((`droit` ou `gauche`) = **état caché**) sur la base d'une **séquence de mesures de vitesse* * (= **observations**).
	- Quatre types d'inférence peuvent être distingués :
		- **Filtrage** : déterminer le **dernier état de croyance**, c'est-à-dire la distribution postérieure p(`voie(t)` | [`vitesse(1)`, ..., `vitesse(t)`]) . Ceci est détaillé dans [Q4](#q4).
		- **Décodage** : déterminer l'état caché **complet** **séquence** qui donne la **meilleure explication** pour l'émission de la séquence d'observation, comme expliqué dans [Q5](#q5).
		- **Prédiction** : déterminer la probabilité de l'**état caché futur en `k` pas**, c'est-à-dire la distribution conditionnelle postérieure p(`voie(t+k)` | [`vitesse(1)`, . .., `vitesse(t)`] ). Il est mentionné dans [Q4](#q4).
		- **Lissage** : déterminez la probabilité pour l'**état caché passé il y a `k` pas**, c'est-à-dire la distribution conditionnelle postérieure p(`voie(t-k)` | [`vitesse(1)`, ... , `vitesse(t)`] ). Il est mentionné dans [Q4](#q4).
		- Avant d'appliquer ces techniques aux séquences, [Q2](#q2) montre comment faire des inférences pour des **observations simples**.

## Hypothèses
Pour garder le problème aussi simple que possible :
- **Discrétisons** la `vitesse` **variable d'observation** en `faible vitesse` et `haute vitesse`.
- Les pas de temps sont également discrétisés.
- Les transitions de voie sont ignorées : soit vous êtes sur la « voie de gauche », soit vous êtes sur la « voie de droite ».

### Processus stationnaire

- On suppose que les modèles HMM (matrice de transition, matrice d'émission) restent **constants dans le temps**.
- p[`vitesse(t)` | `voie(t)`] et p[`voie(t+1)` | `voie(t)`] sont indépendants de `t`.

### Observation Indépendance

- Nous avons parlé de probabilité d'émission, expliquant que l'état `voie(t)` impacte l'observation `vitesse(t)` émise au même pas de temps (`t`).
	- On pourrait imaginer d'autres sources d'influence : « vitesse(t-1) » et « voie(t-1) » par exemple.
- Ici, nous supposons que la **probabilité d'une observation ne dépend que de l'état qui a produit l'observation** et non de tout autre état ou de toute autre observation.
	- En d'autres termes, chaque variable d'observation "vitesse" ne dépend que de l'état courant "voie".
	- Il s'agit d'une **hypothèse forte** puisque nous décidons de ne pas capturer les dépendances directes entre chaque élément de la séquence d'observation.
	- Mais cela va massivement **relâcher le calcul** pendant l'inférence.
- La **probabilité conditionnelle conjointe** suivante peut être simplifiée :
	- p(`vitesse(t)` | `voie(1)` ... `voie(t)`, `vitesse(1)` ... `vitesse(t-1)`) = p(`vitesse( t)` | `voie(t)`).

### Propriété de Markov de premier ordre

- Nous venons de dire qu'il est utile de connaître la `voie` présente (au temps `t`) pour en déduire la future `voie` (au temps `t+1`).

- Voici une hypothèse forte sur l'inférence dans ce processus stochastique :
	- La distribution de probabilité conditionnelle des **états futurs** (conditionnellement aux états passés et présents) **dépend uniquement de l'état présent**, pas de la séquence d'événements qui l'ont précédé.
- En d'autres termes, **"le futur est indépendant du passé étant donné le présent"**.
- Cette **hypothèse forte** est connue sous le nom de **Propriété de Markov** de premier ordre (également appelée **"propriété sans mémoire"**) et facilitera nos calculs.

Sur la base de ces hypothèses, le problème peut être modélisé à l'aide d'un **modèle graphique** :
- Les HMM sont des **modèles orientés** (d'où des flèches) puisqu'on peut distinguer quelle est la raison (état de la voie) et quel est le résultat (observation de la vitesse).
- La **Propriété de Markov** implique des connexions entre états consécutifs.
- L'**indépendance de sortie** fait que chaque observation ne reçoit qu'un seul front (provenant de l'état associé).

| ![Représentation graphique HMM.](docs/hmm_graphical_model.PNG "Représentation graphique HMM.")  | 
|:--:| 
| *Représentation graphique HMM.* |

# Formulation du problème

## Définition :

Un modèle discret de Markov caché (HMM) est un **5-tuple** composé de :

- Un ensemble d'**États cachés** : variable aléatoire discrète `lane` dans {`right_lane`, `left_lane`}.
- Un ensemble d'**Observations** possibles : variable aléatoire discrète `speed` dans {`low_speed`, `high_speed`}.
- Une matrice stochastique qui donne des **probabilités d'émission** : p[`speed(t)` | `lane(t)`].
- Une matrice stochastique qui donne des **probabilités de transition** : p[`lane(t+1)` | `lane(t)`].
- Une distribution **Probabilité d'état initial** : p[`lane(t=1)`].

# Les questions

- [Q1](#q1) - Comment **estimer les paramètres** de notre HMM ?
- [Q2](#q2) - Étant donné une **observation unique de « vitesse »**, quelle est la probabilité que la voiture se trouve dans chacune des deux voies ?
- [Q3](#q3) - Quelle est la probabilité d'observer une **séquence particulière de mesures de « vitesse »** ?
- [Q4](#q4) - Étant donné une **séquence d'observations de « vitesse »**, quelle est la **« voie »** actuelle la plus probable ?
- [Q5](#q5) - Étant donné une **séquence d'observations de « vitesse »**, quelle est la **séquence de « voies » sous-jacente la plus probable** ?
- [Q6](#q6) - Comment **estimer les paramètres** de notre HMM lorsque **aucune annotation "d'état"** n'est présente dans les données d'entraînement ?

# Réponses



*   [Q1](q1)- On estime par la méthode brownien



In [4]:
from tp1_functions import *

Interations:  9


In [5]:
import numpy as np

"""
Compte tenu des observations
"""
O = np.array([[0,1,0,2]]) #observation en séquence
T = len(O[0]) #nombre d'observations en séquence
    
"""
Initialiser les matrices
"""
A = np.array([[.7,.3],
              [.4,.6]]) 
B = np.array([[.1,.4,.5],
              [.7,.2,.1]]) 
pi = np.array([.6,.4]) 

"""
Limites globales
"""
N = np.shape(B)[0]
M = np.shape(B)[1]

In [6]:
A,B,pi,alpha,beta,gamma,digam=markov(O,N)
pi

Interations:  7


array([1.00000000e+00, 1.42727781e-70])

In [9]:
print(A)
print("="*20)
print(B)

[[9.87390041e-30 1.00000000e+00]
 [1.00000000e+00 2.19964880e-60]]
[[1.00000000e+00 2.45100940e-57 9.87390041e-30]
 [1.09982439e-60 5.00000000e-01 5.00000000e-01]]


In [10]:
p_state(gamma)

array([0, 1, 0, 1])